# **Carga de librerías y obtención de los datos desde Kaggle**

En esta celda se realiza todo el proceso necesario para descargar y cargar los datos del concurso.

- Se importan las librerías básicas utilizadas en el proyecto.
- Se verifica si el archivo `kaggle.json` está configurado; si no, se solicita subirlo y se habilitan los permisos necesarios.
- Se descargan los archivos del concurso desde Kaggle usando el nombre de la competencia.
- Se extraen los datos dentro de la carpeta `data/`.
- Finalmente, se cargan los archivos principales: `train.csv`, `test.csv` y el archivo de ejemplo de submission.
- Se imprimen los tamaños de cada dataset y se muestra una vista inicial del conjunto de entrenamiento.

Esta celda garantiza que los datos queden correctamente descargados, organizados y listos para iniciar el análisis.


In [1]:
# === 1. Cargar librerías ===
import os
import pandas as pd
import numpy as np

# === 2. Cargar kaggle.json si no existe ===
if not os.path.exists("/root/.kaggle/kaggle.json"):
    from google.colab import files
    print("Sube aquí tu archivo kaggle.json")
    uploaded = files.upload()

    if "kaggle.json" not in uploaded:
        raise ValueError("Debes subir un archivo llamado kaggle.json")

    os.makedirs("/root/.kaggle", exist_ok=True)
    with open("/root/.kaggle/kaggle.json", "wb") as f:
        f.write(uploaded["kaggle.json"])

    os.chmod("/root/.kaggle/kaggle.json", 600)
    print("kaggle.json configurado correctamente.")

# === 3. Descargar datos del concurso ===
COMP = "udea-ai-4-eng-20252-pruebas-saber-pro-colombia"
!kaggle competitions download -c $COMP --force

# === 4. Extraer archivos ===
os.makedirs("data", exist_ok=True)
!unzip -o {COMP}.zip -d data/

# === 5. Cargar datasets desde la carpeta /data ===
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")
sample_sub = pd.read_csv("data/submission_example.csv")

print("Tamaños de los datos:")
print("Train:", train.shape)
print("Test:", test.shape)
print("Sample submission:", sample_sub.shape)

train.head()


Sube aquí tu archivo kaggle.json


Saving kaggle.json to kaggle.json
kaggle.json configurado correctamente.
 84% 25.0M/29.9M [00:00<00:00, 255MB/s]
100% 29.9M/29.9M [00:00<00:00, 278MB/s]
Archive:  udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip
  inflating: data/submission_example.csv  
  inflating: data/test.csv           
  inflating: data/train.csv          
Tamaños de los datos:
Train: (692500, 21)
Test: (296786, 20)
Sample submission: (296786, 2)


,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


# **Separación de variables predictoras y variable objetivo**

En esta celda se organiza la estructura básica del modelo dividiendo los datos en sus componentes principales.

- Se define la columna objetivo (`RENDIMIENTO_GLOBAL`) y la columna identificadora (`ID`).
- Se guarda el ID del conjunto de prueba para reutilizarlo en la generación del archivo de submission.
- En el conjunto de entrenamiento, se eliminan el ID y la variable objetivo para formar la matriz de características `X`, mientras que `y` contiene exclusivamente el valor a predecir.
- En el conjunto de prueba, también se elimina la columna ID para dejar únicamente las variables de entrada.

De esta forma, los datos quedan correctamente preparados para las etapas de preprocesamiento y modelado.




In [2]:
# === 4. Separar características y variable objetivo ===

target_col = 'RENDIMIENTO_GLOBAL'
id_col = 'ID'

# Guardar el ID del test
test_ids = test[id_col]

# Eliminar ID y la variable objetivo del train
X = train.drop(columns=[id_col, target_col])
y = train[target_col]

# Eliminar ID del test
X_test = test.drop(columns=[id_col])

print("Columnas de entrada:", X.shape[1])


Columnas de entrada: 19


### **Llenado de valores faltantes**

En esta etapa se manejaron los valores faltantes para asegurar que todas las variables pudieran ser utilizadas por los modelos:

- Para las variables numéricas, los valores faltantes se reemplazaron utilizando la media calculada únicamente a partir del conjunto de entrenamiento.
- Para las variables categóricas, los valores faltantes se completaron con la moda, también obtenida del conjunto de entrenamiento.

### **Codificación de variables categóricas**

Se transformaron todas las variables categóricas a valores numéricos para que los modelos pudieran procesarlas.  
Para evitar problemas con categorías presentes solo en el conjunto de prueba, se combinaron temporalmente los datos de entrenamiento y prueba al momento de ajustar el codificador.

### **División del conjunto de datos**

Finalmente, el conjunto de datos fue dividido en entrenamiento y validación de la siguiente manera:

- 80% para entrenamiento  
- 20% para validación  
- Se utilizó muestreo estratificado para mantener la distribución original de la variable objetivo.

Se verificó que ambos conjuntos conservaran sus dimensiones correctas para continuar con el entrenamiento del modelo.


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# === 5. Llenar valores faltantes ===

# Numéricas -> reemplazar con la media del train
numeric_cols = X.select_dtypes(include=np.number).columns

for col in numeric_cols:
    mean_value = X[col].mean()
    X[col] = X[col].fillna(mean_value)
    X_test[col] = X_test[col].fillna(mean_value)

# Categóricas -> reemplazar con la moda del train
categorical_cols = X.select_dtypes(exclude=np.number).columns

for col in categorical_cols:
    moda = X[col].mode()[0]
    X[col] = X[col].fillna(moda)
    X_test[col] = X_test[col].fillna(moda)

# === 6. Codificación robusta de variables categóricas ===
for col in categorical_cols:
    le = LabelEncoder()
    combined = pd.concat([X[col], X_test[col]], axis=0).astype(str)
    le.fit(combined)

    X[col] = le.transform(X[col].astype(str))
    X_test[col] = le.transform(X_test[col].astype(str))

# === 7. División train/validación ===
X_train, X_val, y_train, y_val = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Conjunto de entrenamiento:", X_train.shape)
print("Conjunto de validación:", X_val.shape)


### **Configuración del modelo**

Se definió un modelo de \textit{Random Forest} con 100 árboles y parámetros estándar para iniciar el proceso de clasificación.

### **Validación cruzada estratificada**

Se estableció una validación cruzada con 3 particiones, manteniendo la proporción de clases en cada fold mediante estratificación.  
Esto permitió evaluar el desempeño del modelo de manera estable y consistente.

### **Entrenamiento por folds**

Para cada fold:
- Se entrenó el modelo con una parte del conjunto de datos.
- Se evaluó su rendimiento con la partición correspondiente de validación.
- Se registró el valor de \textit{accuracy} obtenido en cada iteración.

### **Resultado final**

Al finalizar, se calculó el promedio de los valores de \textit{accuracy} obtenidos en cada fold, representando así el desempeño general del modelo bajo validación cruzada.


In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# === 8. Configuración del modelo ===
rf = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)

# === 9. Configurar validación cruzada estratificada ===
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

accuracies = []

# === 10. Entrenamiento y validación por folds ===
for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
    print(f"Entrenando fold {fold}...")

    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    rf.fit(X_train, y_train)
    acc = rf.score(X_val, y_val)
    accuracies.append(acc)

    print(f"   Accuracy del fold {fold}: {acc:.4f}")

# === 11. Promedio final ===
print("\nAccuracy promedio en validación cruzada:", np.mean(accuracies))


### **Entrenamiento del modelo final**

Se entrenó un modelo de Random Forest utilizando todos los parámetros definidos previamente, aplicándolo sobre el conjunto de entrenamiento generado en la partición final.

### **Evaluación del modelo**

Se realizaron predicciones sobre el conjunto de validación y se calculó el \textit{accuracy} para medir el rendimiento.  
Además, se generó el reporte de clasificación para revisar el comportamiento del modelo en cada clase.


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# === 12. Crear y entrenar el modelo final ===
rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=None,
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)

# === 13. Evaluación ===
y_pred = rf.predict(X_val)
acc = accuracy_score(y_val, y_pred)

print(f"Accuracy en validación: {acc:.4f}")
print("\nReporte de clasificación:")
print(classification_report(y_val, y_pred))


### **Procesamiento y entrenamiento del modelo**

Se separaron las variables del objetivo y se aplicó una codificación por frecuencia a las columnas categóricas para representar cada categoría según su proporción en el conjunto de datos.

Posteriormente, se dividieron los datos en entrenamiento y validación utilizando una partición estratificada.

Se entrenó un modelo Random Forest con 100 árboles y se evaluó su rendimiento en el conjunto de validación empleando el accuracy y el reporte de clasificación.


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

# === Usar el dataset 'train' ya cargado desde Kaggle ===

# Separar variables y objetivo
X = train.drop(columns=["RENDIMIENTO_GLOBAL"])
y = train["RENDIMIENTO_GLOBAL"]

# === Codificación ligera de frecuencia ===
for col in X.select_dtypes(include=["object"]).columns:
    freq = X[col].value_counts() / len(X)
    X[col] = X[col].map(freq)

# === División en entrenamiento y validación ===
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# === Entrenamiento del modelo ===
rf = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train, y_train)

# === Evaluación en validación ===
y_pred = rf.predict(X_val)

print("Accuracy en validación:", accuracy_score(y_val, y_pred))
print("\n", classification_report(y_val, y_pred))


### **Generación del archivo de submission**

Se cargaron los datos de prueba y se aplicó la misma codificación por frecuencia utilizada durante el entrenamiento para mantener la consistencia entre ambos conjuntos.

Luego, se generaron las predicciones del modelo sobre el conjunto de test y se construyó el archivo final de submission con las columnas requeridas por la competencia.

Finalmente, el archivo se guardó en la ruta indicada para su posterior envío a Kaggle.


In [ ]:
# === Cargar archivo de test y sample desde /content/data ===
test = pd.read_csv("/content/data/test.csv")
sample_sub = pd.read_csv("/content/data/submission_example.csv")

# === Aplicar la misma codificación de frecuencia al test ===
for col in test.select_dtypes(include=["object"]).columns:
    if col in X.columns:
        freq = X[col].value_counts() / len(X)
        test[col] = test[col].map(freq)
    else:
        test[col] = 0  # Si no existe en X, llenamos con 0

# === Predecir ===
pred_labels = rf.predict(test)

# === Crear archivo de submission ===
submission = pd.DataFrame({
    "ID": sample_sub["ID"],
    "RENDIMIENTO_GLOBAL": pred_labels
})

# === Guardar archivo de submission en /content ===
OUT_SUB = "/content/submission_rf_ligero.csv"
submission.to_csv(OUT_SUB, index=False)

print("Archivo guardado en:", OUT_SUB)
submission.head()


### **Verificación del archivo de submission**

Se comprobó que el archivo de submission se encontrara en la ruta indicada y que su tamaño fuera correcto.  
Luego, se cargó el archivo para revisar sus primeras filas y validar que la estructura coincidiera con el formato requerido por la competencia.  
Finalmente, se imprimió la información general del archivo para confirmar tipos de datos y posibles inconsistencias.


In [ ]:
from pathlib import Path
import pandas as pd
import os

# === Ruta correcta del archivo de submission ===
submission_path = Path("/content/submission_rf_ligero.csv")

# === Verificar que el archivo exista ===
if not submission_path.exists():
    raise FileNotFoundError(f"No se encontró el archivo de submission: {submission_path}")

# === Tamaño del archivo en MB ===
file_size = submission_path.stat().st_size / (1024**2)
print(f"Tamaño del archivo: {file_size:.2f} MB")

# === Cargar archivo ===
submission_df = pd.read_csv(submission_path)

# === Mostrar primeras filas ===
print("Primeras filas del archivo:")
display(submission_df.head())

# === Información general ===
print("\nInformación del archivo:")
print(submission_df.info())


In [ ]:
# === 1. Crear directorio para Kaggle y configurar credenciales ===
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# === 2. Copiar el archivo de submission al nombre requerido por Kaggle ===
!cp /content/submission_rf_ligero.csv submission.csv

# === 3. Enviar a la competencia ===
!kaggle competitions submit \
    -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia \
    -f submission.csv \
    -m "RandomForest ligero - versión reproducible"
